# AFL Predictions
The file we will run to get each rounds predictions!

In [17]:
#packages
import polars as pl
import pickle
import numpy as np
import pandas as pd
#Get Update function
from ipynb.fs.defs.AFLupdate import updateData, getCurrentRoundNumbers
#Get upcoming fixture
from ipynb.fs.defs.AFLfixture import createFixture

pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_cols(40);
pl.Config.set_tbl_rows(50);


In [18]:
#Run updates
updateData()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

No Update
No update Required


In [19]:
#get the fixture 
fixture, predictionDF = createFixture()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 1, 2025"
✔ Returning data for "Round 1, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 0, 2025"
✔ Returning data for "Round 0, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 216 matches.
✔ Finding player stats for 216 matches. [23.6s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [22s]



In [20]:
#if fixture exists i.e no issues have been introduced, go ahead and use our models to get our predictions
modelWin = pickle.load(open('Data/winner.sav', 'rb'))

predictWinner = modelWin.predict(fixture).tolist()
probsWinner = modelWin.predict_proba(fixture).tolist()

#create the winner portion of the prediction df
predictionDF = (predictionDF
                 #place in results
                 .with_columns(pl.Series(predictWinner).alias('Predictions'),
                               pl.Series(probsWinner).alias('Probs'))
                 #Winner probs
                 .with_columns(((pl.col('Probs').list.get(pl.col('Predictions')))*100).round(2).alias('Win Prob'))
                 .with_columns(pl.when(pl.col('Predictions') == 1).then(pl.col('home.team.name')).otherwise(pl.col('away.team.name')).alias('Predicted Winner'))
                 #Add current round
                 .with_columns(pl.lit(getCurrentRoundNumbers()).alias('Round Number'))
                 #rename
                 .rename({'home.team.name' : 'Home Team',
                          'away.team.name' : 'Away Team'})
                 #keep only wants required
                 .select('Round Number', 'Home Team', 'Away Team', 'Predicted Winner', 'Win Prob')
                )


ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]



In [21]:
#Get the margin
#load the model in
model = pickle.load(open('Data/margin.sav', 'rb'))

#And the win probablities to fixture
fixture[['homeProb', 'awayProb']] = probsWinner
#predict the fixture
predictMargin = model.predict(fixture).tolist()

#Add to prediction df
predictionDF = (predictionDF.with_columns(pl.Series(predictMargin).round(0).cast(pl.Int32).alias('Margin')))


In [22]:
predictionDF

Round Number,Home Team,Away Team,Predicted Winner,Win Prob,Margin
i32,str,str,str,f64,i32
1,"""Richmond""","""Carlton""","""Richmond""",66.49,28
1,"""Hawthorn""","""Essendon""","""Hawthorn""",70.7,41
1,"""Geelong Cats""","""Fremantle""","""Geelong Cats""",61.99,39
1,"""Sydney Swans""","""Brisbane Lions""","""Sydney Swans""",68.32,26
1,"""Western Bulldogs""","""North Melbourne""","""North Melbourne""",62.71,27
1,"""Collingwood""","""Port Adelaide""","""Collingwood""",50.39,28
1,"""Adelaide Crows""","""St Kilda""","""Adelaide Crows""",59.68,28
1,"""Melbourne""","""GWS Giants""","""GWS Giants""",51.24,10
1,"""West Coast Eagles""","""Gold Coast Suns""","""West Coast Eagles""",51.94,26


In [ ]:
#Write to folder
#Individual round tips
predictionDF.write_csv(fr'Tips/2025/Round_{getCurrentRoundNumbers()}.csv')

#Append to all_tips
with open("Tips/2025/All_Tips.2025.csv", mode="a") as f:
   predictionDF.write_csv(f, include_header=False)

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

